In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [6]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='5um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 11:02AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:02AM [load_ansys_project]: 	Opened Ansys App
INFO 11:02AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:02AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project1
INFO 11:02AM [connect_design]: No active design found (or error getting active design).
INFO 11:02AM [connect]: 	 Connected to project "Project1". No design detected


True

In [14]:
hfss.activate_ansys_design("flip5um_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

11:03AM 10s WARNING [activate_ansys_design]: The design_name=flip5um_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:03AM [connect_design]: 	Opened active design
	Design:    flip5um_V1 [Solution type: Eigenmode]
WARNING 11:03AM [connect_setup]: 	No design setup detected.
WARNING 11:03AM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:03AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [15]:
hfss.close()

True

# LOM Analysis

In [19]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [20]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='7.80',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '7.80',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [22]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:10PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpf_s1oeja.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 7800000000, Maxwell, 1, False
INFO 06:10PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp_cx5zj1h.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 7800000000, Maxwell, 1, False
INFO 06:10PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp75razbe4.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 7800000000, Maxwell, 2, False
INFO 06:10PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpmlchssac.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 7800000000, Maxwell, 3, False
INFO 06:10PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwwf13nu6.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 7800000000, Maxwell, 4, False
INFO 06:10

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,428.59710,-148.07170,-145.92130,-68.50343
pad_bot_Q0,-148.07170,176.35990,-27.53427,-0.25498
pad_top_Q0,-145.92130,-27.53427,176.48680,-2.71986
quad_coupler_c0,-68.50343,-0.25498,-2.71986,71.69015


In [23]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 10.05
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.515655 [GHz]
EC 186.655771 [MHz]
EJ 14.854145 [GHz]
alpha -207.163253 [MHz]
dispersion 0.072832 [KHz]
Lq 10.995564 [nH]
Cq 103.775137 [fF]
T1 35036.183461 [us]

**Coupling Properties**

tCqbus1 -1.230883 [fF]
gbus1_in_MHz -18.976417 [MHz]
χ_bus1 -0.005409 [MHz]
1/T1bus1 4.542588 [Hz]
T1bus1 35036.183461 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.369975,268.864221,14.854145,-306.215425,4.345006,[-56.289863043321596],[-0.09148295373085212],0.091483,56.289863
2,4.897877,221.380449,14.854145,-248.474665,0.542501,[2.7356142543409643],[-0.0001505831795811813],0.000151,2.735614
3,4.695474,202.584751,14.854145,-226.017303,0.195217,[-8.19235894503028],[-0.0011569982681865255],0.001157,8.192359
4,4.57372,191.720861,14.854145,-213.140657,0.100999,[-18.85914917389264],[-0.00558561558453369],0.005586,18.859149
5,4.528394,187.760637,14.854145,-208.465706,0.078305,[-13.388206839554739],[-0.0027187565405105978],0.002719,13.388207
6,4.521259,187.141367,14.854145,-207.735593,0.075195,[-14.185495574381216],[-0.003035537592759574],0.003036,14.185496
7,4.526929,187.633421,14.854145,-208.315699,0.077657,[-15.87513556001298],[-0.0038183178166052768],0.003818,15.875136
8,4.518956,186.94176,14.854145,-207.50031,0.074216,[-18.4283403763886],[-0.005113888197790728],0.005114,18.428340
9,4.515655,186.655771,14.854145,-207.163253,0.072832,[-18.976416653040413],[-0.005408866161789012],0.005409,18.976417


In [24]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
